# Análisis de Prices y Currencys
## 0. Imports, carga de datos y funciones
Lo primero para poder ordenar el campo price y currency deberiamos considerar que las propiedades en Argentina se comercializan en dólares por usos y costumbre.

Por lo que la prioridad siempre será tener el precio expresado en esa moneda y determinar una tasa de cambio a la moneda local.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
# Función que devuelve un diccionario con unos valores estadisticos que quería ver
def getQs(df, listaLugares,column):
    dicc = {}
    for lugar in listaLugares:
        
        #armo un subset por cada lugar
        #se podría agregar un try catch acá para validar la columna
        subset = df[df['place_with_parent_names'] == lugar].loc[:,column]

        #calculo los Q y otras medidas del subset y los agrego al diccionario
        q = subset.quantile([0.25,0.75])
        dicc[lugar] = {'min':round(subset.min(),2),'0.25': round(q[0.25],2), 'media': round(subset.mean(),2),'0.75': round(q[0.75],2), 'max':round(subset.max(),2),'iiq': round(q[0.75]-q[0.25],2),'linf':round(-1.5*q[0.75]+2.5*q[0.25],2),'lsup':round(2.5*q[0.75]-1.5*q[0.25],2)}
    return dicc

In [ ]:
#Funciones para limpiar el findall

def sacarValores(serie, cambio):
    listReemplazo = []
    for x in serie:
        if len(x)==0:
            listReemplazo.append(np.NaN)
        else:
            listReemplazo.append(listaConPlata(x,cambio))
    return listReemplazo

def listaConPlata(lista,cambio):
    controlUSD = 0.00
    controlARS = 0.00
    for x in lista:
        if (x[0].strip() == 'USD') | (x[0].strip() == 'U$D'):
            #algo en USD
            string = x[1].replace('.','').replace(',','')
            try:
                priceUSD = float(string)
            except:
                print(string, x[1])
            controlUSD = getMax(controlUSD,priceUSD)
        elif x[0].strip() == 'ARS':
            #algo en ARS
            string = x[1].replace('.','').replace(',','')
            try:
                priceARS = float(string)
            except:
                print(string, x[1])
            controlARS = getMax(controlARS,priceARS)
        else:
            np.NaN
        maxPriceUSD = getMax(controlUSD,controlARS*cambio)
    # Siempre devuelvo el mayor porque siempre va a ser el más próximo al precio de mercado
    return maxPriceUSD

def getMax(A,B):
    valor = 0
    if A >= B:
        valor = A
    else:
        valor = B
    return valor

In [ ]:
# Se carga el csv y valido los null
properati = pd.read_csv('./properati.csv', dtype={'operation' : 'category','property_type' : 'category','place_name' : 'category','country_name' : 'category','state_name' : 'category','currency' : 'category'})


In [ ]:
# Cambio el nombre de la primer columna que viene sin nombre
properati = properati.rename(columns={'Unnamed: 0':'ID'})
properati.dtypes

In [ ]:
# Total de registros
print(f"En total el set tiene {properati['ID'].count()} filas")

totalRegistros = properati['ID'].count()

## 1. Homogeneizar Monedas
Lo más lógico sería comprender la distribución en base a que monedas componen los precios y tratar de llevar todos sus valores a única moneda y siendo que el dólar es la moneda habitual lo mejor sería expresarlo en dólares.

In [ ]:
print(f"Las cantidad por currency es las siguiente:\n{properati.currency.value_counts()}")

De la muestra se detecta que PEN y UYU parecieran un error de carga y que dado su cantidad son candidatos a ser eleminados.

Con los precios en ARS se hará una tranformación a dólares en base a la tasa de cambio derivada de la columna price_aprox_local_currency/price para los price con currency en USD.

In [ ]:
# Comprobamos el describe de la tasa de cambio
(properati['price_aprox_local_currency'].loc[properati['currency']=='USD']/properati['price'].loc[properati['currency']=='USD']).describe().apply(lambda x: round(x,2))


Podemos ver que la tasa 17.64 es única para todas las propiedades expresadas en dólares y por ende es el valor que tomaremos para tranformar la columna price a price_usd que usaremos en adelante.

In [ ]:
# Me quedo por la media porque en todo caso es lo mismo que el ~= al máximo y en caso de un año sería mejor
tasaARSUSD = (properati['price_aprox_local_currency'].loc[properati['currency']=='USD']/properati['price'].loc[properati['currency']=='USD']).mean()
tasaARSUSD

In [ ]:
# Genero la columna priceUSD
properati['priceUSD'] = properati.apply(lambda x: round(x['price'] / tasaARSUSD,0) if x['currency'] == 'ARS' else x['price'], axis=1)


In [ ]:
properati[['price','price_aprox_local_currency','currency','priceUSD']].loc[properati['currency']=='ARS']

## 2. Buscar con Regex el precio en la descripción
Una vez realizada la unificación de la monedad se procedera a buscar dentro de las descripciones cualquier monto expresado en ARS o USD y también se lo va a unificar a dólares.

Para eso el primer paso será armar la expresión regular por la cual encontrar dichos valores.

In [ ]:
# Armo el pattern para encontrar los montos en las descripciones
pattern = '(?P<currency>U\$D|ARS\s*\$|USD\s*\$?)\s*(?P<price>\d{1,3}(?:\.?\d{3})*(?:\,\d+)*)'
pattern_regex = re.compile(pattern)

# Lo fuerzo a string porque me dio un error pero no creo que sea necesario
descripcion = properati.description.astype('str')

# Busco todas las coincidencias posibles
resultado = descripcion.apply(lambda x: pattern_regex.findall(x))

In [ ]:
# Llamo a la función que arme para descomponer la lista de listas que devuelve el findall
regexPriceUSD = sacarValores(resultado,tasaARSUSD)

#Lo convierto en serie para unir y pisar properati.priceUSD
impRegex = pd.Series(regexPriceUSD)

# Uno lo que generó el Regex al DF de properati
properati['impRegex'] = impRegex

# Esta es la cantidad de nulos que deberían quedar
print(properati.loc[(properati['priceUSD'].isnull())]['ID'].count()-properati.loc[(properati['priceUSD'].isnull())&(~properati['impRegex'].isnull())]['ID'].count())


In [ ]:
# En las filas vacias de properati['priceUSD'] le imputo el valor que devolvió el Regex
properati['priceUSD'] = properati[['priceUSD','impRegex']].apply(lambda x: x['impRegex'] if pd.isnull(x['priceUSD']) else x['priceUSD'], axis=1)

properati.priceUSD.isnull().sum()

## 3. Análisis de Nulls en los campos [priceUSD]
Siendo que restan imputar 19956 valores a pricesUSD se proceda a trata de completar los registros pendientes con otro metodo.

In [ ]:
print(f"La cantidad de nulos en las columnas [priceUSD] es de: {properati['priceUSD'].isnull().sum()}")

In [ ]:
#almaceno la serie de true/false de los price nulls
filtroPriceUSDNull = properati.priceUSD.isnull()
filtroPriceUSDNull.sum()

In [ ]:
#Almaceno en un dataframe solo los valores nulos para saber como afrontarlo
soloNulos = properati[filtroPriceUSDNull]
soloNulos.shape[0] == properati['priceUSD'].isnull().sum()

Por el volumen de nulls sería importante simplificar el problema para darle prioridad a los casos y poder simplificar la imputación. Para eso, lo primero que podemos considerar es como se comportan ante la ubicación por lo que a continuación trataremos de determinar si se comportan según el principio de Pareto:

In [ ]:
nulosPorUbicacion = soloNulos.groupby(['place_with_parent_names'])['place_with_parent_names'].count()
paretoNulos = nulosPorUbicacion.to_frame()
paretoNulos.columns = ['cantidad']
paretoNulos = paretoNulos.sort_values(by= 'cantidad', ascending=False)
paretoNulos['pareto'] = paretoNulos['cantidad'].cumsum()/paretoNulos['cantidad'].sum()*100

print(f"El 80% de los casos se acumula en el {round((paretoNulos[paretoNulos['pareto'] <=80].shape[0]/paretoNulos.shape[0])*100,ndigits=2)}% de las ubicaciones")


Por lo que podemos ver está mucho más concetrado que la proporción de Pareto por lo que si nos concentramos en estos lugares podriamos lograr completar más nulls con el menor análisis posible y comprender si es replicable en los demás.

In [ ]:
# Creo una lista de las ubicaciones que tienen más nulos
maskUbicacionConMasNulos = paretoNulos[paretoNulos['pareto'] <=80].index

# Creo un dataframe con los registros de las ubicaciones más nulas que tienen precio
conPrecioMasUbicacionesNulas = properati[(~filtroPriceUSDNull) & (properati.place_with_parent_names.isin(maskUbicacionConMasNulos))]

# Los agrupo para despues unirlo al análisis de pareto
paraJoinConPareto = conPrecioMasUbicacionesNulas.groupby(['place_with_parent_names'])['place_with_parent_names'].count()
dfParaJoinConPareto = paraJoinConPareto.to_frame()
dfParaJoinConPareto.columns = ['cantidadConPrecio']

Uno el agrupado de los que más nulos tienen con la cantidad de precios que tienen esas mismas ubicaciones en el dataset original.

La idea es imputar por la media los nulos de cada ubicacion, pero para mitigar el efecto adverso de reducir la variabilidad solo lo voy a aplicar sobre las regiones donde por cada nulo tenga más de 10 precios reales.

In [ ]:
# Armo el DF con las columnas de cantidad de nulos por región y la cantidad de precios de esa región
ratiosNulosSobreConPrecio = paretoNulos.join(dfParaJoinConPareto)

# Creo el ratios de precio por nulo
ratiosNulosSobreConPrecio['ratioSinConPrice'] = (ratiosNulosSobreConPrecio['cantidadConPrecio']/ratiosNulosSobreConPrecio['cantidad'])

# Muestro el impacto de la correción
print(f"Se podrían corregir por este metodo {ratiosNulosSobreConPrecio[ratiosNulosSobreConPrecio['ratioSinConPrice'] > 10].cantidad.sum()} nulls")


In [ ]:
# Detalle de ubicaciones a corregir por imputación de media
ratiosNulosSobreConPrecio[ratiosNulosSobreConPrecio['ratioSinConPrice'] > 10]

In [ ]:
# Este sería el grupo donde el ratio es mayor que el limite que fijamos de tolerancia
ratiosNulosSobreConPrecio[ratiosNulosSobreConPrecio['ratioSinConPrice'] <= 10].sort_values(by='ratioSinConPrice')

De los casos donde por cada nulo tenemos menos de 10 precios originales podemos ver que hay regiones donde practicamente no hay precios originales por lo que tendremos que buscar otra manera de imputarlas o estimarlas.

Llegado el caso que no posamos hacerlo no nos va a quedar más que dar de baja esos nulos.

In [ ]:
# Creo una lista con los lugares que voy a analizar
listaLugares = ratiosNulosSobreConPrecio[ratiosNulosSobreConPrecio['ratioSinConPrice'] > 10].index

# Armo un diccionario de control
dccQs = getQs(conPrecioMasUbicacionesNulas,listaLugares,'priceUSD')
pd.DataFrame(dccQs)

In [ ]:
# Por cada lugar grafico su distribución
for lugar in listaLugares:
    # Grupo de cada lugar
    # Sería bueno meter esto en la función de getQs
    subset = conPrecioMasUbicacionesNulas[conPrecioMasUbicacionesNulas['place_with_parent_names'] == lugar].priceUSD
    # Armo el distplot por cada subset que arme en base a los lugares
    #sns.set(rc={"figure.figsize": (16, 8)})
    sns.set_style("dark")
    sns.distplot(subset, hist = False, kde = True,
                 kde_kws = {'linewidth': 3},
                 label = lugar)
    # Formateo
    plt.legend(prop={'size': 9}, title = 'Lugares')
    plt.title('Distribución de los precios')
    plt.xlabel('Precio')
    plt.figure()




### 3.1. Primeras concluciones de las distribuciones para la imputación por media

De los valores estadisticos descriptivos y de la gráfica de las distribuciones (aproximadas) podes detectar que tenemos una gran cantidad de ouliers por defecto y por exceso.

Hablamos por defecto ya que los valores minimos por cada zona están muy por debajo de lo que podría ser un valor razonable. Por ejemplo si tenemos en cuanto el código de edificación de la C.A.B.A [(Link al código)](http://www2.cedom.gob.ar/es/legislacion/normas/codigos/edifica/index3a.html) la vivienda mínima es de 20.5 M2 de superficie cubierta, siendo este la menor superficie aprobable en todo el país, y si tenemos en cuenta que el precio promedio del M2 en toda la C.A.B.A [(Link a data set de USDxM2 de 2AMB y 3AMB)](https://data.buenosaires.gob.ar/dataset/mercado-inmobiliario/archivo/c6d2a64a-f60b-4b6e-9829-919139a0c1d1) es de ~ USD 2.535 por M2 es razonable esperar que los precios minimos de cualquier propiedad ronden por lo menos entorno de los ~ USD 51.000 un valor muy seprior a los minimos de muchas regiones en cuestión.

Los valores por exceso son más complejos ya que pueden ser lugares con muchos M2 o en zonas donde el M2 es muy alto o ambas. Pero queda claro que esto convierte estas propiedades en excepciones (en las gráficas se vé que las tails de la distribución no suman mucha frequencia) y por lo tanto tampoco sería convenientes considerarlas para una imputación por media.



In [ ]:
# Genero un diccionario para la correción
diccCorrecion = {}
for lugar in listaLugares:
    algo = properati.loc[properati.place_with_parent_names == lugar]
    aca = algo.loc[(properati.priceUSD >= dccQs[lugar]['linf']) & (properati.priceUSD >= dccQs[lugar]['lsup'])]
    diccCorrecion[lugar] = round(aca.priceUSD.mean(),2)
    
diccCorrecion

In [ ]:
# Imputo con un apply los valores de la media que tenía en el diccionario
properati['priceUSDImp'] = properati.apply(lambda x: diccCorrecion[x['place_with_parent_names']] if (x['place_with_parent_names'] in diccCorrecion.keys()) & (pd.isnull(x['priceUSD'])) else x['priceUSD'] ,axis = 1)


In [ ]:
properati.loc[(properati['priceUSD'].isnull())&(~properati['priceUSDImp'].isnull())]

In [ ]:
#Armo el diccionario para comprar controlar los cambios en la distribución
dccPosterior = getQs(properati,listaLugares,'priceUSDImp')
dccControl = getQs(properati,listaLugares,'priceUSD')
pd.DataFrame(dccPosterior)

In [ ]:
pd.DataFrame(dccControl)

In [ ]:
pd.DataFrame(dccControl)-pd.DataFrame(dccPosterior)

In [ ]:
properati.columns

In [ ]:
print(f"La nueva cnatidad de nulos en la columnas priceUSD debería ser {properati['priceUSDImp'].isnull().sum()}")

In [ ]:
# En las filas vacias de properati['priceUSD'] le imputo el valor que devolvió la media de las zonas
properati['priceUSD'] = properati[['priceUSD','priceUSDImp']].apply(lambda x: x['priceUSDImp'] if pd.isnull(x['priceUSD']) else x['priceUSD'], axis=1)

properati[['price','priceUSD']].isnull().sum()

## 4. Análisis de mínimos
En este punto evaluaremos el impacto de realizar lo que se planteo en el punto 3.1. respecto de los valores mínimos.

In [ ]:
# Armo un DF solo con los precios en dóalres y su ubicación
analisisMinimos = properati[['priceUSD','place_with_parent_names']]

# Pivoteo el DF para que cada columna sea una ubicación
pivotAnalisisMinimos = analisisMinimos.pivot(columns='place_with_parent_names',values='priceUSD')

In [ ]:
# Compruebo que se mantenga la proporción
print(pivotAnalisisMinimos.shape)

# Obtengo el describe de cada ubicación
describe = pivotAnalisisMinimos.describe()

In [ ]:
# Obtengo la cantidad de filas candidatas
print(pivotAnalisisMinimos[pivotAnalisisMinimos <= 51000].count().sum())

#
print(describe.loc['std'].max())

## 5. Concluciones de la limpieza

Se logró reducir el nro de nulos de 20.410 (\~16% del total) a 18.806 (\~15% del total), pero nos permitio detectar muchas puntos de mejora y carasteristicas del dataset a la largo del análisis.

**1.** Las distribuciones dentro de las zonas parecieran ser muy dispersas.
   * Sería conveniente aperturar por tipo de propiedad para mejorar la imputación por media reduciendo la variabilidad por zona.
   * Sería prudente descartar los valores que no se aproximen a un precio mínimo razonable ya que valores de USD 1 solo perjudican la estimación.

**2.** Si bien los barrios difieren mucho entre si todas las distribuciones presentan un fuerte sesgo hacia a la izquierda.
   * Esto nos habla que hay una regla en común los precios más bajos son los que acumulan mayores cantidades.
   * Se podría evaluar hacer un regresor del precio en base a los m2 y barrio (minimamente) para reemplazar la imputación por media y por Regex.
   
**3.** Si tomamos como valor mínimo USD 51.000 se perderían ~4.000 registros.
   * Esto no sería de gran impacto sobre el total (\~4% del total) y ayudaría a disminuir el inmenso std que se observa por barrio.